# get_rawdata 함수 만들기

In [14]:
import pandas as pd
import requests
import random
import time
from tqdm import tqdm
from my_utils import *

In [15]:
riot_api_key = 'RGAPI-96f2dd58-1f36-4e86-a50a-cb61514088fd'

In [16]:
# 강사님 코드
def get_puu_id(summoner_id, riot_api_key):
    url2 = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={riot_api_key}'
    res2 = requests.get(url2).json()
    puu_id = res2['puuid']
    return puu_id

In [17]:
def get_match_ids(puu_id, riot_api_key, page=20):
    url3 = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?start=0&count={page}&api_key={riot_api_key}'
    res3 = requests.get(url3).json()
    return res3

In [18]:
def get_matches_timeLines(match_ids, riot_api_key):
    match_info_list = []
    for match_id in tqdm(match_ids):
        url4 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={riot_api_key}'
        url5 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={riot_api_key}'
        res4 = requests.get(url4).json()
        res5 = requests.get(url5).json()
        match_info_list.append([match_id,res4,res5])
    return match_info_list

In [19]:
def get_rawdata(tier):
    # 원하는 티어를 넣으면 해당 디비전에 속한 특정 몇명의 사람들만 뽑는 로직
    lst = []
    divisions = ['I','II','III','IV']
    page = random.randrange(1,51) # range처럼 1부터 50까지 나옴 -> 실행할때마다 1~50 사이의 숫자중에 램덤으로 하나가 나온다
    print('get Summoner_id ..... ')
    for division in tqdm(divisions):
        url = f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={riot_api_key}'
        res = requests.get(url).json()
        lst += random.sample(res, 3) # res안에서 랜덤으로 3개 뽑아서 lst에 넣어라 => 리스트랑 어떤 요소를 +연산자를 사용하면 extend효과가 난다
    summoner_id_list = list(map(lambda x: x['summonerId'], lst))
    
    # 위에서 뽑은 사람들의 puu_id를 가져오는 로직
    print('get Puu_id ..... ')
    puu_id_list = []
    for summoner_id in tqdm(summoner_id_list):
        puu_id = get_puu_id(summoner_id, riot_api_key)
        puu_id_list.append(puu_id)

    # puu_id들을 통해 match_id들을 뽑는 로직
    print('get Match_id ..... ')
    match_id_list = []
    for p in tqdm(puu_id_list):
        match_ids = get_match_ids(p, riot_api_key, 2)
        match_id_list.extend(match_ids) # 이중리스트가 안되고 그냥 리스트와 리스트를 합치려면 extend를 사용한다

    # match_id를 통해 match_id/matches/timLines가 들어있는 데이터 프레임 만들기
    print('get matches & timeLines ..... ')
    df_create = get_matches_timeLines(match_id_list, riot_api_key)
    df = pd.DataFrame(df_create, columns=['match_id','matches','timeline'])
    
    return df

# match  - ['match_id','gameDuration','gameVersion','summonerName','summonerLevel','participantId','championName', 'champExperience','teamPosition','teamId','win','kills','deaths','assists', 'totalDamageDealtToChampions','totalDamageTaken']


# timeline - timeline_df ['g5','g6' ~ 'g25'] -> 25분까지 게임이 있으면 0음 0

# return -> df

In [20]:
def get_match_timeline_df(df):
    df_create = []
    print('소환사 스텟 생성중.....')
    for i in tqdm(range(len(df))):
        try:
            if df.iloc[i].matches['info']['gameDuration'] > 600:#게임이 10분 보다 오래됐는지 확인
                for j in range(10):
                    tmp = []
                    tmp.append(df.iloc[i].match_id)
                    info = df.iloc[i].matches['info']
                    tmp.append(info['gameDuration'])
                    tmp.append(info['gameVersion'])
                    tmp.append(info['participants'][j]['riotIdGameName'])
                    tmp.append(info['participants'][j]['riotIdTagline'])
                    tmp.append(info['participants'][j]['summonerLevel'])
                    tmp.append(info['participants'][j]['participantId'])
                    tmp.append(info['participants'][j]['championName'])
                    tmp.append(info['participants'][j]['champExperience'])
                    tmp.append(info['participants'][j]['teamPosition'])
                    tmp.append(info['participants'][j]['teamId'])
                    tmp.append(info['participants'][j]['win'])
                    tmp.append(info['participants'][j]['kills'])
                    tmp.append(info['participants'][j]['deaths'])
                    tmp.append(info['participants'][j]['assists'])
                    tmp.append(info['participants'][j]['totalDamageDealtToChampions'])
                    tmp.append(info['participants'][j]['totalDamageTaken'])
    
                    # timelines뽑아낼 G5~ 26까지의 데이터
                    for k in range(5,26):
                        try:
                            tmp.append(df.iloc[i].timeline['info']['frames'][k]['participantFrames'][str(j+1)]['totalGold'])
                        except:
                            tmp.append(0)
                    df_create.append(tmp)
        except Exception as e:
            print(e)
            continue
    columns = ['gameId','gameDuration','gameVersion','riotIdGameName','riotIdTagline','summonerLevel','participantId','championName','champExperience',
    'teamPosition','teamId','win','kills','deaths','assists','totalDamageDealtToChampions','totalDamageTaken','g_5','g_6','g_7','g_8','g_9','g_10','g_11','g_12','g_13','g_14','g_15','g_16','g_17',
    'g_18','g_19','g_20','g_21','g_22','g_23','g_24','g_25']
    df= pd.DataFrame(df_create,columns =columns).drop_duplicates()
    print('현재 df의 수는 %d 입니다.'% len(df))
    return df

In [21]:
query = """
create table lol_datas(gameId varchar(30), gameDuration number(20), gameVersion varchar(40), riotIdGameName varchar (100),
riotIdTagline varchar(30),summonerLevel number(30), participantId number(5), championName varchar(50), champExperience number(20),
teamPosition varchar(50), teamId number(20), win varchar(50), kills number(20), deaths number(20), assists number(20),
totalDamageDealtToChampions number(30) ,totalDamageTaken number(30), g_5 number(20),g_6 number(20),g_7 number(20),
g_8 number(20),g_9 number(20),g_10 number(20),g_11 number(20),g_12 number(20),g_13 number(20),g_14 number(20),
g_15 number(20),g_16 number(20),g_17 number(20),g_18 number(20),g_19 number(20),g_20 number(20),g_21 number(20),
g_22 number(20),g_23 number(20),g_24 number(20),g_25 number(20),
constraint m_pk primary key(gameId,participantId)
)
"""

In [22]:
db_open()
sql_execute(query)
db_close()

oracle 쿼리 성공
오라클 정상적으로 닫힘


In [23]:
def insert_data(x):
    query =(
    f"MERGE INTO lol_datas USING DUAL ON(gameId = {repr(x.gameId)} and participantId = {x.participantId})"
    f"WHEN NOT MATCHED THEN "
    f"insert (gameId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,"
    f"summonerLevel,participantId,championName,champExperience,teamPosition,"
    f"teamId,win,kills,deaths,assists,totalDamageDealtToChampions,totalDamageTaken,"
    f"g_5,g_6,g_7,g_8,g_9,g_10,g_11,g_12,g_13,g_14,g_15,g_16,g_17,g_18,g_19,g_20,"
    f"g_21,g_22,g_23,g_24,g_25) values ({repr(x.gameId)},{x.gameDuration},{repr(str(x.gameVersion))},"
    f"{repr(x.riotIdGameName)},{repr(x.riotIdTagline)},{x.summonerLevel},{x.participantId},"
    f"{repr(x.championName)},{x.champExperience},{repr(x.teamPosition)},{x.teamId},"
    f"{repr(str(x.win))}, {x.kills},{x.deaths},{x.assists},{x.totalDamageDealtToChampions},"
    f"{x.totalDamageTaken},{x.g_5},{x.g_6},{x.g_7},{x.g_8},{x.g_9},{x.g_10},{x.g_11},{x.g_12},{x.g_13},{x.g_14},"
    f"{x.g_15},{x.g_16},{x.g_17},{x.g_18},{x.g_19},{x.g_20},{x.g_21},{x.g_22},{x.g_23},{x.g_24},{x.g_25})"
    )
    sql_execute(query)

In [ ]:
tqdm.pandas()
for i in range(100000000000):
    try:
        db_open()
        tier = 'SILVER'
        df = get_rawdata(tier)
        result_df=get_match_timeline_df(df)
        result_df.progress_apply(lambda x :insert_data(x) ,axis=1) #insert 코드
        db_close()
        time.sleep(60) #riot api는 2분에 100개
    except Exception as e:
        print(e)
        time.sleep(60)
        continue
        

get Summoner_id ..... 


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.30it/s]


get Puu_id ..... 


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.54it/s]


get Match_id ..... 


...

oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공


 92%|████████████████████████████████████████████████████████████████████████▋      | 221/240 [00:01<00:00, 136.76it/s]

oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공


100%|███████████████████████████████████████████████████████████████████████████████| 240/240 [00:01<00:00, 124.10it/s]


oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
oracle 쿼리 성공
오라클 정상적으로 닫힘
